In [1]:
!pip install 2048

In [2]:
import pygame, os
import _2048
from _2048.game import Game2048
from _2048.manager import GameManager

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import os, pygame, time, random, math
from copy import deepcopy
from pprint import pprint
import numpy as np
import _2048
from _2048.game import Game2048
from _2048.manager import GameManager

# define events
EVENTS = [
  pygame.event.Event(pygame.KEYDOWN, {'key': pygame.K_UP}),   # UP
  pygame.event.Event(pygame.KEYDOWN, {'key': pygame.K_RIGHT}), # RIGHT
  pygame.event.Event(pygame.KEYDOWN, {'key': pygame.K_DOWN}), # DOWN
  pygame.event.Event(pygame.KEYDOWN, {'key': pygame.K_LEFT}) # LEFT
]

CELLS = [
  [(r, c) for c in range(4) for r in range(4)], # UP
  [(r, c) for r in range(4) for c in range(4 - 1, -1, -1)], # RIGHT
  [(r, c) for c in range(4) for r in range(4 - 1, -1, -1)], # DOWN
  [(r, c) for r in range(4) for c in range(4)], # LEFT
]

GET_DELTAS = [
  lambda r, c: ((i, c) for i in range(r + 1, 4)), # UP
  lambda r, c: ((r, i) for i in range(c - 1, -1, -1)), # RIGHT
  lambda r, c: ((i, c) for i in range(r - 1, -1, -1)), # DOWN
  lambda r, c: ((r, i) for i in range(c + 1, 4)) # LEFT
]

def free_cells(grid):
  return [(x, y) for x in range(4) for y in range(4) if not grid[y][x]]

def move(grid, action):
  moved, sum = 0, 0
  for row, column in CELLS[action]:
    for dr, dc in GET_DELTAS[action](row, column):
      # If the current tile is blank, but the candidate has value:
      if not grid[row][column] and grid[dr][dc]:
        # Move the candidate to the current tile.
        grid[row][column], grid[dr][dc] = grid[dr][dc], 0
        moved += 1
      if grid[dr][dc]:
        # If the candidate can merge with the current tile:
        if grid[row][column] == grid[dr][dc]:
          grid[row][column] *= 2
          grid[dr][dc] = 0
          sum += grid[row][column]
          moved += 1
        # When hitting a tile we stop trying.
        break
  return grid, moved, sum

def evaluation(grid, n_empty):
  grid = np.array(grid)

  score = 0

  # grid sum
  big_t = np.sum(np.power(grid, 2))

  # smoothness
  smoothness = 0
  s_grid = np.sqrt(grid)

  smoothness -= np.sum(np.abs(s_grid[:, 0] - s_grid[:, 1]))
  smoothness -= np.sum(np.abs(s_grid[:, 1] - s_grid[:, 2]))
  smoothness -= np.sum(np.abs(s_grid[:, 2] - s_grid[:, 3]))
  smoothness -= np.sum(np.abs(s_grid[0, :] - s_grid[1, :]))
  smoothness -= np.sum(np.abs(s_grid[1, :] - s_grid[2, :]))
  smoothness -= np.sum(np.abs(s_grid[2, :] - s_grid[3, :]))

  # monotonicity
  monotonic_up = 0
  monotonic_down = 0
  monotonic_left = 0
  monotonic_right = 0

  for x in range(4):
    current = 0
    next = current + 1
    while next < 4:
      while next < 3 and not grid[next, x]:
        next += 1
      current_cell = grid[current, x]
      current_value = math.log(current_cell, 2) if current_cell else 0
      next_cell = grid[next, x]
      next_value = math.log(next_cell, 2) if next_cell else 0
      if current_value > next_value:
        monotonic_up += (next_value - current_value)
      elif next_value > current_value:
        monotonic_down += (current_value - next_value)
      current = next
      next += 1

  for y in range(4):
    current = 0
    next = current + 1
    while next < 4:
      while next < 3 and not grid[y, next]:
        next += 1
      current_cell = grid[y, current]
      current_value = math.log(current_cell, 2) if current_cell else 0
      next_cell = grid[y, next]
      next_value = math.log(next_cell, 2) if next_cell else 0
      if current_value > next_value:
        monotonic_left += (next_value - current_value)
      elif next_value > current_value:
        monotonic_right += (current_value - next_value)
      current = next
      next += 1

  monotonic = max(monotonic_up, monotonic_down) + max(monotonic_left, monotonic_right)
  
  # weight for each score
  empty_w = 100000
  smoothness_w = 3
  monotonic_w = 10000

  empty_u = n_empty * empty_w
  smooth_u = smoothness ** smoothness_w
  monotonic_u = monotonic * monotonic_w

  score += big_t
  score += empty_u
  score += smooth_u
  score += monotonic_u

  return score

def maximize(grid, depth=0):
  best_score = -np.inf
  best_action = None

  for action in range(4):
    moved_grid = deepcopy(grid)
    moved_grid, moved, _ = move(moved_grid, action=action)

    if not moved:
      continue

    new_score = add_new_tiles(moved_grid, depth+1)
    if new_score >= best_score:
      best_score = new_score
      best_action = action

  return best_action, best_score

def add_new_tiles(grid, depth=0):
  fcs = free_cells(grid)
  n_empty = len(fcs)

  # early stopping
  if n_empty >= 6 and depth >= 3:
    return evaluation(grid, n_empty)

  if n_empty >= 0 and depth >= 5:
    return evaluation(grid, n_empty)

  if n_empty == 0:
    _, new_score = maximize(grid, depth+1)
    return new_score

  sum_score = 0

  for x, y in fcs:
    for v in [2, 4]:
      new_grid = deepcopy(grid)
      new_grid[y][x] = v

      _, new_score = maximize(new_grid, depth+1)

      if v == 2:
        new_score *= (0.9 / n_empty)
      else:
        new_score *= (0.1 / n_empty)

      sum_score += new_score

  return sum_score

def run_game(game_class=Game2048, title='2048!', data_dir='save'):
  pygame.init()
  pygame.display.set_caption(title)
  pygame.display.set_icon(game_class.icon(32))
  clock = pygame.time.Clock()

  os.makedirs(data_dir, exist_ok=True)

  screen = pygame.display.set_mode((game_class.WIDTH, game_class.HEIGHT))
  # screen = pygame.display.set_mode((50, 20))
  manager = GameManager(Game2048, screen,
              os.path.join(data_dir, '2048.score'),
              os.path.join(data_dir, '2048.%d.state'))

  # faster animation
  manager.game.ANIMATION_FRAMES = 1
  manager.game.WIN_TILE = 999999

  # game loop
  tick = 0
  running = True
  
  while running:
    clock.tick(120)
    tick += 1

    if tick % 5 == 0:
      old_grid = deepcopy(manager.game.grid)

      best_action, best_score = maximize(old_grid)

      if best_action is None:
        print('No way! Maximum number is %s' % np.max(manager.game.grid))
        break

      print(best_action)
      e = EVENTS[best_action]
      manager.dispatch(e)
      pprint(manager.game.grid, width=30)
      print(manager.game.score)

    for event in pygame.event.get():
      if event.type == pygame.QUIT:
        running = False
      elif event.type == pygame.MOUSEBUTTONUP:
        manager.dispatch(event)
    
    manager.draw()
  # end while

  pygame.quit()
  manager.close()


In [4]:
run_game()

Running as instance #0.
3
[[0, 2, 0, 0],
 [2, 0, 0, 0],
 [0, 0, 0, 0],
 [4, 0, 0, 0]]
0
0
[[2, 2, 0, 0],
 [4, 0, 0, 0],
 [0, 0, 0, 0],
 [2, 0, 0, 0]]
0
3
[[4, 0, 0, 0],
 [4, 0, 0, 0],
 [0, 0, 0, 0],
 [2, 0, 2, 0]]
4
3
[[4, 0, 0, 0],
 [4, 2, 0, 0],
 [0, 0, 0, 0],
 [4, 0, 0, 0]]
8
0
[[8, 2, 0, 0],
 [4, 0, 0, 0],
 [0, 0, 0, 2],
 [0, 0, 0, 0]]
16
0
[[8, 2, 0, 2],
 [4, 0, 0, 0],
 [0, 0, 0, 0],
 [2, 0, 0, 0]]
16
1
[[2, 0, 8, 4],
 [0, 0, 0, 4],
 [0, 0, 0, 0],
 [0, 0, 0, 2]]
20
0
[[2, 0, 8, 8],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 0, 0, 0]]
28
1
[[0, 0, 2, 16],
 [0, 0, 2, 2],
 [0, 0, 0, 2],
 [0, 0, 0, 0]]
44
2
[[0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 16],
 [0, 2, 4, 4]]
52
1
[[0, 0, 0, 0],
 [0, 0, 0, 0],
 [2, 0, 0, 16],
 [0, 0, 2, 8]]
60
0
[[2, 0, 2, 16],
 [0, 0, 0, 8],
 [0, 0, 0, 0],
 [0, 2, 0, 0]]
60
1
[[0, 0, 4, 16],
 [0, 0, 0, 8],
 [2, 0, 0, 0],
 [0, 0, 0, 2]]
64
1
[[0, 0, 4, 16],
 [0, 0, 0, 8],
 [0, 2, 0, 2],
 [0, 0, 0, 2]]
64
2
[[0, 0, 0, 0],
 [0, 0, 0, 16],
 [0, 0, 2, 8],
 [0, 2, 4, 4]]
6

1
[[0, 2, 0, 2],
 [0, 0, 0, 64],
 [0, 4, 16, 128],
 [0, 4, 32, 16]]
1272
1
[[4, 0, 0, 4],
 [0, 0, 0, 64],
 [0, 4, 16, 128],
 [0, 4, 32, 16]]
1276
1
[[0, 0, 0, 8],
 [0, 2, 0, 64],
 [0, 4, 16, 128],
 [0, 4, 32, 16]]
1284
2
[[0, 0, 0, 8],
 [0, 0, 0, 64],
 [0, 2, 16, 128],
 [2, 8, 32, 16]]
1292
0
[[2, 2, 16, 8],
 [0, 8, 32, 64],
 [0, 0, 2, 128],
 [0, 0, 0, 16]]
1292
1
[[0, 4, 16, 8],
 [0, 8, 32, 64],
 [0, 0, 2, 128],
 [0, 2, 0, 16]]
1296
0
[[0, 4, 16, 8],
 [0, 8, 32, 64],
 [0, 2, 2, 128],
 [2, 0, 0, 16]]
1296
0
[[2, 4, 16, 8],
 [0, 8, 32, 64],
 [2, 2, 2, 128],
 [0, 0, 0, 16]]
1296
0
[[4, 4, 16, 8],
 [0, 8, 32, 64],
 [2, 2, 2, 128],
 [0, 0, 0, 16]]
1300
1
[[0, 8, 16, 8],
 [2, 8, 32, 64],
 [0, 2, 4, 128],
 [0, 0, 0, 16]]
1312
0
[[2, 16, 16, 8],
 [0, 2, 32, 64],
 [0, 0, 4, 128],
 [0, 0, 2, 16]]
1328
1
[[0, 2, 32, 8],
 [0, 2, 32, 64],
 [0, 0, 4, 128],
 [0, 2, 2, 16]]
1360
1
[[2, 2, 32, 8],
 [0, 2, 32, 64],
 [0, 0, 4, 128],
 [0, 0, 4, 16]]
1364
2
[[0, 2, 0, 8],
 [0, 0, 0, 64],
 [0, 0, 64, 128],

0
[[2, 2, 2, 32],
 [4, 4, 4, 64],
 [0, 16, 16, 256],
 [0, 2, 32, 64]]
2736
1
[[0, 2, 4, 32],
 [2, 4, 8, 64],
 [0, 0, 32, 256],
 [0, 2, 32, 64]]
2780
2
[[0, 0, 0, 32],
 [0, 2, 4, 64],
 [4, 4, 8, 256],
 [2, 2, 64, 64]]
2844
1
[[0, 0, 0, 32],
 [2, 2, 4, 64],
 [0, 8, 8, 256],
 [0, 0, 4, 128]]
2984
1
[[0, 0, 0, 32],
 [0, 4, 4, 64],
 [2, 0, 16, 256],
 [0, 0, 4, 128]]
3004
1
[[0, 0, 0, 32],
 [0, 0, 8, 64],
 [0, 2, 16, 256],
 [2, 0, 4, 128]]
3012
0
[[2, 2, 8, 32],
 [0, 0, 16, 64],
 [0, 2, 4, 256],
 [0, 0, 0, 128]]
3012
2
[[0, 0, 0, 32],
 [0, 2, 8, 64],
 [0, 0, 16, 256],
 [2, 4, 4, 128]]
3016
1
[[0, 0, 0, 32],
 [0, 2, 8, 64],
 [0, 2, 16, 256],
 [0, 2, 8, 128]]
3024
2
[[0, 0, 0, 32],
 [0, 0, 8, 64],
 [2, 2, 16, 256],
 [0, 4, 8, 128]]
3028
1
[[2, 0, 0, 32],
 [0, 0, 8, 64],
 [0, 4, 16, 256],
 [0, 4, 8, 128]]
3032
2
[[0, 2, 0, 32],
 [0, 0, 8, 64],
 [0, 0, 16, 256],
 [2, 8, 8, 128]]
3040
1
[[2, 0, 2, 32],
 [0, 0, 8, 64],
 [0, 0, 16, 256],
 [0, 2, 16, 128]]
3056
1
[[0, 0, 4, 32],
 [0, 0, 8, 64],
 [2,

1
[[0, 2, 16, 512],
 [0, 0, 32, 128],
 [0, 2, 2, 32],
 [0, 0, 4, 16]]
5116
1
[[0, 2, 16, 512],
 [0, 0, 32, 128],
 [0, 0, 4, 32],
 [4, 0, 4, 16]]
5120
1
[[0, 2, 16, 512],
 [0, 0, 32, 128],
 [0, 0, 4, 32],
 [2, 0, 8, 16]]
5128
1
[[0, 2, 16, 512],
 [0, 0, 32, 128],
 [0, 2, 4, 32],
 [0, 2, 8, 16]]
5128
0
[[0, 4, 16, 512],
 [2, 2, 32, 128],
 [0, 0, 4, 32],
 [0, 0, 8, 16]]
5132
2
[[0, 0, 16, 512],
 [0, 0, 32, 128],
 [2, 4, 4, 32],
 [2, 2, 8, 16]]
5132
1
[[0, 0, 16, 512],
 [2, 0, 32, 128],
 [0, 2, 8, 32],
 [0, 4, 8, 16]]
5144
2
[[0, 0, 0, 512],
 [0, 0, 16, 128],
 [2, 2, 32, 32],
 [2, 4, 16, 16]]
5160
1
[[0, 0, 0, 512],
 [0, 0, 16, 128],
 [0, 0, 4, 64],
 [2, 2, 4, 32]]
5260
0
[[2, 2, 16, 512],
 [0, 0, 8, 128],
 [0, 2, 0, 64],
 [0, 0, 0, 32]]
5268
1
[[0, 4, 16, 512],
 [2, 0, 8, 128],
 [0, 0, 2, 64],
 [0, 0, 0, 32]]
5272
0
[[2, 4, 16, 512],
 [0, 0, 8, 128],
 [0, 0, 2, 64],
 [2, 0, 0, 32]]
5272
0
[[4, 4, 16, 512],
 [0, 0, 8, 128],
 [0, 0, 2, 64],
 [0, 0, 2, 32]]
5276
1
[[0, 8, 16, 512],
 [0, 0, 8

0
[[2, 8, 16, 512],
 [0, 8, 64, 256],
 [0, 0, 16, 64],
 [4, 0, 2, 32]]
6648
1
[[2, 8, 16, 512],
 [0, 8, 64, 256],
 [0, 0, 16, 64],
 [2, 4, 2, 32]]
6648
0
[[4, 16, 16, 512],
 [2, 4, 64, 256],
 [0, 0, 16, 64],
 [0, 0, 2, 32]]
6668
1
[[0, 4, 32, 512],
 [2, 4, 64, 256],
 [2, 0, 16, 64],
 [0, 0, 2, 32]]
6700
0
[[4, 8, 32, 512],
 [0, 0, 64, 256],
 [0, 2, 16, 64],
 [0, 0, 2, 32]]
6712
2
[[0, 0, 32, 512],
 [0, 2, 64, 256],
 [0, 8, 16, 64],
 [4, 2, 2, 32]]
6712
1
[[0, 0, 32, 512],
 [0, 2, 64, 256],
 [2, 8, 16, 64],
 [0, 4, 4, 32]]
6716
1
[[0, 0, 32, 512],
 [0, 2, 64, 256],
 [2, 8, 16, 64],
 [0, 2, 8, 32]]
6724
0
[[2, 2, 32, 512],
 [2, 8, 64, 256],
 [0, 2, 16, 64],
 [0, 0, 8, 32]]
6724
1
[[0, 4, 32, 512],
 [2, 8, 64, 256],
 [0, 2, 16, 64],
 [2, 0, 8, 32]]
6728
0
[[4, 4, 32, 512],
 [2, 8, 64, 256],
 [0, 2, 16, 64],
 [0, 0, 8, 32]]
6732
1
[[0, 8, 32, 512],
 [2, 8, 64, 256],
 [4, 2, 16, 64],
 [0, 0, 8, 32]]
6740
0
[[2, 16, 32, 512],
 [4, 2, 64, 256],
 [0, 2, 16, 64],
 [0, 0, 8, 32]]
6756
2
[[0, 2, 

2
[[2, 0, 0, 1024],
 [0, 0, 0, 128],
 [0, 0, 16, 32],
 [4, 4, 4, 4]]
10000
1
[[0, 0, 2, 1024],
 [0, 2, 0, 128],
 [0, 0, 16, 32],
 [0, 0, 8, 8]]
10016
1
[[0, 0, 2, 1024],
 [2, 0, 2, 128],
 [0, 0, 16, 32],
 [0, 0, 0, 16]]
10032
2
[[0, 0, 0, 1024],
 [0, 0, 0, 128],
 [0, 2, 4, 32],
 [2, 0, 16, 16]]
10036
2
[[0, 0, 0, 1024],
 [4, 0, 0, 128],
 [0, 0, 4, 32],
 [2, 2, 16, 16]]
10036
1
[[0, 0, 0, 1024],
 [0, 0, 4, 128],
 [0, 0, 4, 32],
 [0, 4, 4, 32]]
10072
1
[[0, 0, 0, 1024],
 [0, 0, 4, 128],
 [2, 0, 4, 32],
 [0, 0, 8, 32]]
10080
0
[[2, 0, 8, 1024],
 [0, 0, 8, 128],
 [2, 0, 0, 64],
 [0, 0, 0, 0]]
10152
1
[[0, 2, 8, 1024],
 [0, 0, 8, 128],
 [0, 0, 2, 64],
 [0, 0, 0, 2]]
10152
0
[[0, 2, 16, 1024],
 [0, 0, 2, 128],
 [0, 0, 2, 64],
 [0, 0, 0, 2]]
10168
0
[[0, 2, 16, 1024],
 [0, 0, 4, 128],
 [0, 0, 0, 64],
 [0, 0, 2, 2]]
10172
1
[[0, 2, 16, 1024],
 [0, 0, 4, 128],
 [0, 0, 0, 64],
 [0, 2, 0, 4]]
10176
2
[[0, 0, 0, 1024],
 [0, 0, 0, 128],
 [2, 0, 16, 64],
 [0, 4, 4, 4]]
10180
1
[[0, 0, 0, 1024],
 [0,

2
[[0, 2, 4, 1024],
 [0, 2, 8, 256],
 [0, 8, 32, 64],
 [0, 8, 16, 32]]
11468
2
[[0, 2, 4, 1024],
 [0, 0, 8, 256],
 [0, 4, 32, 64],
 [0, 16, 16, 32]]
11488
2
[[0, 0, 4, 1024],
 [0, 2, 8, 256],
 [2, 4, 32, 64],
 [0, 16, 16, 32]]
11488
2
[[0, 0, 4, 1024],
 [2, 2, 8, 256],
 [0, 4, 32, 64],
 [2, 16, 16, 32]]
11488
2
[[2, 0, 4, 1024],
 [0, 2, 8, 256],
 [0, 4, 32, 64],
 [4, 16, 16, 32]]
11492
2
[[0, 0, 4, 1024],
 [2, 2, 8, 256],
 [2, 4, 32, 64],
 [4, 16, 16, 32]]
11492
1
[[0, 0, 4, 1024],
 [0, 4, 8, 256],
 [2, 4, 32, 64],
 [2, 4, 32, 32]]
11528
0
[[4, 8, 4, 1024],
 [0, 4, 8, 256],
 [0, 0, 64, 64],
 [0, 2, 0, 32]]
11604
0
[[4, 8, 4, 1024],
 [0, 4, 8, 256],
 [0, 2, 64, 64],
 [0, 2, 0, 32]]
11604
1
[[4, 8, 4, 1024],
 [0, 4, 8, 256],
 [0, 0, 2, 128],
 [2, 0, 2, 32]]
11732
2
[[0, 0, 0, 1024],
 [2, 0, 4, 256],
 [4, 8, 8, 128],
 [2, 4, 4, 32]]
11736
1
[[0, 0, 0, 1024],
 [2, 2, 4, 256],
 [0, 4, 16, 128],
 [0, 2, 8, 32]]
11760
1
[[0, 0, 0, 1024],
 [0, 4, 4, 256],
 [2, 4, 16, 128],
 [0, 2, 8, 32]]
1176